In [3]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from collections import Counter
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re
from vis_utils import * 


/home/jessica/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using concept features


In [4]:
for j, batch in enumerate(dataloader_comma):
    _, image_array, vego, angle, distance, m_lens, i_lens, s_lens, a_lens, d_lens = batch
    img = image_array
    max_len = 5
    print(image_array.shape)
    img, angle, distance, vego = img.to(gpu)[:, 0:10], angle.to(gpu)[:, 0:10], distance.to(gpu)[:, 0:10], vego.to(gpu)[:, 0:10]
    (logits, attns), concepts = model(img, angle, distance, vego)
    top5_indices = torch.tensor(concepts.squeeze()).topk(5).indices
    s = img.shape
    angle, distance, vego, logits, concepts = angle.to("cpu"), distance.to("cpu"), vego.to("cpu"), logits.detach().cpu().to("cpu"), concepts.detach().cpu().to("cpu")

    atten = attns[0][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph = alignment_array.sum(axis=0)[8:-8]
    speed_graph_0 = moving_average(speed_graph, 5)

    atten = attns[1][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph = alignment_array.sum(axis=0)[8:-8]
    speed_graph_1 = moving_average(speed_graph, 5)

    atten = attns[2][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph = alignment_array.sum(axis=0)[8:-8]
    speed_graph_2 = moving_average(speed_graph, 5)
    speed_graph = speed_graph_0 + speed_graph_1 + speed_graph_2
    
    f = []
    inter = []
    for i, elem in enumerate(top5_indices):
        if i % 20 == 0:
            inter = []
        inter.extend(elem.cpu().numpy().tolist())
        count_dict = Counter(inter)

        # Get the top 5 most occurring numbers
        top_5 = count_dict.most_common(3)
        intermediate = []
        for a in top_5: 
            intermediate.append(scenarios[a[0]])
        f.append(intermediate)

    
    for i, image in enumerate(img[0]): 
        image = unorm(image).cpu()
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))

        # Assuming you have an image frame
        image_frame = image.permute(1,2,0)

        # Display the image frame
        ax1.imshow(image_frame)
        ax1.set_title('\n'.join([h.replace("a photo of driving on a highway with", "").replace("a photo of", "") for h in f[i]]))
        ax1.set_aspect('equal')
        ax1.set_xticks([])
        ax1.set_yticks([])

        # Remove borders
        ax1.spines['top'].set_visible(False)
        ax1.spines['bottom'].set_visible(False)
        ax1.spines['left'].set_visible(False)
        ax1.spines['right'].set_visible(False)

        # Plot the speed graph
        ax2.plot(speed_graph_1, label='Attention')
        #ax2.plot(speed_graph_2, label='Attention Head 2')
        #ax2.plot(speed_graph_0, label='Attention Head 3')
        ax2.set_title("Attention")

        
        ax2.plot(i, speed_graph_0[i], marker='o', markersize=10, color='r')
        #ax2.plot(i, speed_graph_1[i], marker='o', markersize=10, color='r')
        #ax2.plot(i, speed_graph_2[i], marker='o', markersize=10, color='r')
        ax1.set_aspect('equal')
        plt.legend()

        # Adjust the plot layout if necessary
        plt.tight_layout()

        # Show the plot
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        plt.savefig(f"/home/jessica/personalized_driving_toyota/result_images/attention/{i}.png")
        plt.clf()
    image_directory = '/home/jessica/personalized_driving_toyota/result_images/attention'

    # Set the output GIF file path
    output_gif_path = f'/home/jessica/personalized_driving_toyota/attention_comma_{j}.gif'

    # Set the duration (in milliseconds) for each frame in the GIF
    frame_duration = 500

    # Get a sorted list of image files in the directory
    image_files = sorted(glob.glob(f'{image_directory}/*.png'), key=extract_number)  # Adjust the file extension if necessary

    # Create a list to store the frames of the GIF
    frames = []

    # Iterate over each image file
    for image_file in image_files:
        # Open the image file
        image = Image.open(image_file)

        # Add the image to the list of frames
        frames.append(image)

    # Save the frames as a GIF
    frames[0].save(output_gif_path, format='GIF', append_images=frames[1:], save_all=True,
                duration=frame_duration, loop=0)
    attention_weights = att
    normalized_weights = attention_weights#np.array(attention_weights) / np.sum(attention_weights)

    fig, ax = plt.subplots(figsize=(10, 6))
    heatmap = ax.pcolormesh(alignment_array[0:70, :], cmap='hot')
    # Add colorbar
    cbar = plt.colorbar(heatmap)

    # Add labels and title
    ax.set_xlabel('Sequence Position')
    ax.set_ylabel('Window')
    ax.set_title('Longformer Sliding Chunk Attention')

    # Show the plot
    plt.savefig(f"/home/jessica/personalized_driving_toyota/attention_vis{j}.png")
    break

torch.Size([1, 240, 3, 224, 224])


OutOfMemoryError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 2; 11.91 GiB total capacity; 10.65 GiB already allocated; 70.19 MiB free; 11.10 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF